#  Flask

- ### Flask es un framework web: Proporciona las herramientas para crear aplicaciones web
- ### Es un microframework: Sin dependencias a librerías externas
- ### Framework ligero

- #### [https://flask.palletsprojects.com/en/stable/](https://flask.palletsprojects.com/en/stable/)
- #### Opciones para el manejo de Bases de Datos
    - #### Adaptadores de base de datos PostgreSQL como Psycopg
        - #### [https://pypi.org/project/psycopg2](https://pypi.org/project/psycopg2)
    - #### Framework ORM (Object Relational Mapper) como SQLAlchemy
        - #### [https://www.sqlalchemy.org](https://www.sqlalchemy.org)


# Instalar Flask
- ### Desde la terminal en el ambiente (TPS), instalar flask con pip 

In [ ]:
pip install flask

### Comprobar la versión instalada de Flask

In [ ]:
import flask
print(flask.__version__)

In [2]:
# Crear el directorio de la aplicación
!mkdir app_web


## Aplicacion Web en FLASK 

- ### Una vez que cree la instancia app, se utiliza para gestionar las solicitudes web entrantes y enviar respuestas al usuario. 

- ###  @app.route es un decorador que convierte una función Python regular en una función vista de Flask

- ###  Convierte el valor de devolución de la función en una respuesta HTTP que se mostrará mediante un cliente HTTP

- ###   Pasa el valor '/' a @app.route() para indicar que esta función responderá a las solicitudes web para la URL /, que es la URL principal.

- ###  La función de vista hello() devuelve la cadena 'Hello, World!'​​ como respuesta.


In [ ]:
# Crear el archivo inicio.py

from flask import Flask

app = Flask(__name__)


@app.route('/')
def inicio():
    return '¡Hola, mundo! Esta es mi primera aplicación Flask.'

if __name__ == '__main__':
    app.run(debug=True)


### Ejecutar la aplicacion


#### Desde la terminal, en el directorio de la aplicación

#### <span style="color:#0485CF"> python inicio.py </span>



```
* Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 947-720-893
127.0.0.1 - - [16/Feb/2024 14:56:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2024 14:56:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Feb/2024 14:58:34] "GET / HTTP/1.1" 200 -
```



#### Otra forma de ejecuta la aplicación web usando el puerto 5000
#### <span style="color:#0485CF"> flask --app inicio.py --debug run -p 5000 </span>



# Entrar al navegador para ver los resultados
## http://127.0.0.1:5000


# Ejemplo de pagina de login con HTML

### 1. Crear la carpeta "templates" en el directorio de la aplicación
### 2. Tener una plantilla HTML para tu página de inicio de sesión


In [ ]:
# templates/login.html:

<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Iniciar sesión</title>
</head>
<body>
    <h1>Iniciar sesión</h1>
    <form action="/login" method="post">
        <label for="username">Usuario:</label><br>
        <input type="text" id="usuario" name="usuario"><br>
        <label for="password">Contraseña:</label><br>
        <input type="password" id="contrasena" name="contrasena"><br><br>
        <input type="submit" value="Iniciar sesión">
    </form>
</body>
</html>

### 3. En la aplicación Flask,  definir una ruta para la página de inicio de sesión y otra para manejar el envío del formulario de inicio de sesión.

In [ ]:
# Archivo inicio.py
from flask import Flask, render_template, request, redirect, url_for


app = Flask(__name__)

# Datos de usuarios (simulados para el ejemplo)
USUARIOS = {
    'usuario1': 'u1',
    'usuario2': 'u2'
}



@app.route('/')
def inicio():
    return '¡Hola, mundo! Esta es mi primera aplicación Flask.'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['usuario']
        password = request.form['contrasena']
        if username in USUARIOS and USUARIOS[username] == password:
            # Iniciar sesión exitosa, redirigir a otra página
            return redirect(url_for('inicio'))
        else:
            # Credenciales incorrectas, mostrar mensaje de error
            return 'Credenciales incorrectas. <a href="/login">Intenta de nuevo</a>'
    else:
        return render_template('login.html')
    


if __name__ == '__main__':
    app.run(debug=True)


# Entrar al navegador para ver los resultados
## http://127.0.0.1:5000/login


## Conectando con la Base de Datos

- #### En Flask, "g" es un objeto que se utiliza para almacenar y compartir datos temporalmente entre aplicaciones. Es útil para gestionar recursos durante una solicitud, como crear un recurso si no existe. 
- #### jsonify es una función de Flask que convierte diccionarios y listas a formato JSON. Se usa para serializar datos y enviarlos como respuestas a solicitudes. 
- #### En Flask, @app.teardown_appcontext es una función que se llama cuando se expulsa el contexto de la aplicación. El contexto de la aplicación se crea para cada solicitud y se utiliza para procesar las solicitudes. 



In [ ]:
# Archivo inicio.py o crear otra aplicacion inicio_db.py
import psycopg2.pool
from flask import Flask, g, jsonify

app = Flask(__name__)


db_config = { "host" : "localhost",
                "database" : "sistema_abc",
                "user" : "uacm",
                "password" : "uacm1"}
# Crear un pool de conexiones

app.config['postgreSQL_pool']  = psycopg2.pool.ThreadedConnectionPool(minconn=1, maxconn=30, **db_config)

def obtener_conexion():
    # Obtener una conexión del pool
    if 'db' not in g:
        g.db = app.config['postgreSQL_pool'].getconn()
    return g.db

@app.teardown_appcontext
def cerrar_conexion(exception):
    print("\n\n")
    print("---"*30)
    print("liberando  conexiones de la BD")
    print("---"*30)
    db = g.pop('db', None)
    if db is not None:
        app.config['postgreSQL_pool'].putconn(db)
    else:
        print("no hay db")


    
@app.route('/consulta')
def consultaBD():
    resultados  = []
    try:
        # Obtener una conexión
        conexion = obtener_conexion()

        # Crear un cursor
        cursor = conexion.cursor()

        # Ejecutar una consulta
        cursor.execute("SELECT * FROM productos")

        # Obtener los resultados
        filas = cursor.fetchall()

        # Mostrar los resultados
        for fila in filas:
            print(fila)
            resultados.append(str(fila)) 

        # Cerrar el cursor
        cursor.close()

    except Exception as e:
        print("Error:", e)

    return "Consulta realizada. Revisa la consola para ver los resultados. \n" + " ".join(resultados)

if __name__ == '__main__':
    app.run(debug=True)

# Entrar al navegador para ver los resultados
## http://127.0.0.1:5000/consulta


### Ejemplo de Consulta y despliegue de los datos en una página HTML

- ### 1. Crear la página de login (login.html)
- ### 2. Crear la página del menú de la aplicación (menu.html)
- ### 3. Crear la aplicación (app_web.py)
- ### 4. Crear la página de consulta de datos (consulta.py)




### Crear Archivo: templates/login.html

In [ ]:
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Iniciar sesión</title>
</head>
<body>
    <h1>Iniciar sesión</h1>
    <form action="/login" method="post">
        <label for="username">Usuario:</label><br>
        <input type="text" id="usuario" name="usuario"><br>
        <label for="password">Contraseña:</label><br>
        <input type="password" id="contrasena" name="contrasena"><br><br>
        <input type="submit" value="Iniciar sesión">
    </form>
</body>
</html>

### Crear Archivo: templates/menu.html

In [ ]:
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Página de Inicio</title>
</head>
<body>
    <h1>Bienvenido al menú de  aplicación</h1>
    <a href="/consulta">Consultar datos</a>
</body>
</html>

### Crear archivo: app_web.py

In [ ]:
import psycopg2.pool
from flask import Flask, g, jsonify, render_template, request, redirect, url_for

app = Flask(__name__)

# Datos de usuarios (simulados para el ejemplo)
USUARIOS = {
    'usuario1': 'u1',
    'usuario2': 'u2'
}


db_config = { "host" : "localhost",
                "database" : "sistema_abc",
                "user" : "uacm",
                "password" : "uacm1"}
# Crear un pool de conexiones

app.config['postgreSQL_pool']  = psycopg2.pool.ThreadedConnectionPool(minconn=1, maxconn=30, **db_config)


#--------------------------------------------------

def obtener_conexion():
    # Obtener una conexión del pool
    if 'db' not in g:
        g.db = app.config['postgreSQL_pool'].getconn()
    return g.db

#--------------------------------------------------


#--------------------------------------------------
@app.teardown_appcontext
def cerrar_conexion(exception):
    print("\n\n")
    print("---"*30)
    print("liberando  conexiones de la BD")
    print("---"*30)
    db = g.pop('db', None)
    if db is not None:
        app.config['postgreSQL_pool'].putconn(db)
    else:
        print("no hay db")
#--------------------------------------------------



#--------------------------------------------------
@app.route('/menu', methods=['GET', 'POST'])
def menu():
        return render_template('menu.html')


#--------------------------------------------------

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['usuario']
        password = request.form['contrasena']
        if username in USUARIOS and USUARIOS[username] == password:
            # Iniciar sesión exitosa, redirigir a otra página
            return redirect(url_for('menu'))
        else:
            # Credenciales incorrectas, mostrar mensaje de error
            return 'Credenciales incorrectas. <a href="/login">Intenta de nuevo</a>'
    else:
        return render_template('login.html')
    

#--------------------------------------------------
@app.route('/consulta', methods=['GET', 'POST'])
def consultaBD():
    try:
        # Obtener una conexión
        conexion = obtener_conexion()
        # Crear un cursor
        cursor = conexion.cursor()
        # Ejecutar una consulta
        cursor.execute("SELECT * FROM productos")
        # Obtener los resultados
        resultados = cursor.fetchall()
        # Cerrar el cursor
        cursor.close()

    except Exception as e:
        print("Error:", e)

    return render_template('consulta.html', datos=resultados)


#--------------------------------------------------


if __name__ == '__main__':
    app.run(debug=True)

### 2. Crear el archivo consulta.html, que es la página que desplegará los datos

- #### En Flask, la etiqueta {% for %} se utiliza para crear bucles for en plantillas.
- #### Las variables se usan con {{ }},


In [ ]:
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Consulta</title>
    
</head>
<body>
    <h1>Lista de Datos</h1>
    <table>
        <tr>
            <th>id</th>
            <th>DESC</th>
            <th>precio</th>
        </tr>
        {% for id, desc, precio in datos %}
        <tr>
            <td>{{ id }}</td>
            <td>{{ desc }}</td>
            <td>{{ precio }}</td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>


# Entrar al navegador para ver los resultados
## http://127.0.0.1:5000/login


# Ejercicio

- #### 1. Hacer una aplicación web que contenga un login de inicio de sesión
- #### 2. Tenga un menú donde se desplieguen las opciones del sistema
- #### 3. Consulte los datos de la tabla Empleados (todos)
- #### 4. Consulte los datos de la tabla Empleados menores a cierto monto


